In [1]:
import sys
sys.path.append("../models")
sys.path.append("../utils")
import config
import time
import numpy as np
import tensorflow as tf
import data_loader
from tra_helper.image_aug import img_missing
from tra_helper.plot_dset_one import plot_dset_one
from base_models.mobilenet import mobilenet_v2
from seg_models.unet import unet
from seg_models.unet_backbone import unet_backbone
from seg_models.deeplabv3_plus import deeplabv3_plus
from seg_models.model_triple import model_triple
from seg_models.unet_triple import unet_triple


### Training configuration

In [2]:
root_dir = '/home/yons/Desktop/developer-luo/SWatNet'
strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
print ('Number of devices: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
Number of devices: 2


In [3]:
## 1. dataset
tra_dset = data_loader.get_tra_dset()
test_dset = data_loader.get_eva_dset()
tra_dset_dist = strategy.experimental_distribute_dataset(tra_dset)
test_dset_dist = strategy.experimental_distribute_dataset(test_dset)

## 2. training configuration
with strategy.scope():
    ## 2.1 loss function
    loss_fun = tf.keras.losses.BinaryCrossentropy(
                from_logits=False,
                reduction=tf.keras.losses.Reduction.NONE)
    def compute_loss(labels, predictions):
        per_example_loss = loss_fun(labels, predictions)
        # per_example_loss /= tf.cast(tf.reduce_prod(tf.shape(labels)[1:]), tf.float32)
        return tf.nn.compute_average_loss(per_example_loss, global_batch_size=config.BATCH_SIZE)
    tra_loss_tracker = config.tra_loss_tracker
    test_loss_tracker = config.test_loss_tracker
    ## 2.2 accuracy metric
    tra_oa = config.tra_oa
    tra_miou = config.tra_miou


### Model loading

In [4]:
## model
with strategy.scope():
    #### single scale
    # model = unet(nclass=2)
    # model = unet_backbone(input_shape=(256,256,4), base_model=mobilenet_v2)
    # model = deeplabv3_plus(input_shape=[256,256,4], base_model=mobilenet_v2, nclasses=2)
    #### multiple scale
    model = model_triple(input_shape=(256,256,4), base_model=mobilenet_v2)
    # model = unet_triple(scale_high=2048, scale_mid=512, scale_low=256, nclass=2)
    optimizer = config.optimizer
    # model.summary()


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [5]:
'''------train step------'''
# @tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        y_pre = model(x, training=True)
        loss = compute_loss(y, y_pre)
    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    tra_loss_tracker.update_state(loss)
    tra_oa.update_state(y, y_pre)
    tra_miou.update_state(y, y_pre)
    return tra_loss_tracker.result(), tra_oa.result(), tra_miou.result()

@tf.function
def distributed_train_step(x,y):
  per_replica_losses = strategy.run(train_step, args=(x,y,))
  return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses,
                         axis=None)

'''------test step------'''
# @tf.function
def test_step(x, y):
    # with tf.GradientTape() as tape:
    y_pre = model(x, training=False)
    loss = loss_fun(y, y_pre)
    test_loss_tracker.update_state(loss)
    test_oa.update_state(y, y_pre)
    test_miou.update_state(y, y_pre)
    return test_loss_tracker.result(), test_oa.result(), test_miou.result()

@tf.function
def distributed_test_step(x,y):
  return strategy.run(test_step, args=(x,y,))

'''------train loops------'''
def train_loops(tra_dset, test_dset, epochs):
    max_miou_pre = 0.8
    for epoch in range(epochs):
        start = time.time()

        '''---train the model---'''
        for x_batch, y_batch in tra_dset:
            # x_batch = img_missing(x_batch)    
            # x_batch=x_batch[2]   ##!!note: x_batch[2] for single-scale model
            tra_loss_epoch,tra_oa_epoch,tra_miou_epoch = train_step(x_batch, y_batch)

        '''---test the model---'''
        for x_batch, y_batch in test_dset:
            # x_batch=x_batch[2]  ##!note: x_batch[2] for single-scale model
            test_loss_epoch, test_oa_epoch, test_miou_epoch = test_step(x_batch, y_batch)
        
        '''---reset the metrics---'''
        tra_loss_tracker.reset_states(), tra_oa.reset_states(), tra_miou.reset_states()
        test_loss_tracker.reset_states(), test_oa.reset_states(), test_miou.reset_states()

        # '''---write into tensorboard---'''
        # train_summary_writer = tf.summary.create_file_writer(config.train_log_dir)
        # test_summary_writer = tf.summary.create_file_writer(config.test_log_dir)
        # with train_summary_writer.as_default():
        #     tf.summary.scalar('learning rate', data=config.optimizer.learning_rate(epoch*16), step=epoch)
        #     tf.summary.scalar('loss', data=tra_loss_epoch, step=epoch)
        #     tf.summary.scalar('oa', data=tra_oa_epoch, step=epoch)
        #     tf.summary.scalar('miou', data=tra_miou_epoch, step=epoch)
        # with test_summary_writer.as_default():
        #     tf.summary.scalar('loss', data=test_loss_epoch, step=epoch)
        #     tf.summary.scalar('oa', data=test_oa_epoch, step=epoch)
        #     tf.summary.scalar('miou', data=test_miou_epoch, step=epoch)
        # print the metrics
        print('epoch {}: traLoss:{:.3f}, traOA:{:.2f}, traMIoU:{:.2f}; evaLoss:{:.3f}, evaOA:{:.2f}, evaMIoU:{:.2f}, time:{:.0f}s'.format(epoch + 1, tra_loss_epoch, tra_oa_epoch, tra_miou_epoch, test_loss_epoch, test_oa_epoch, test_miou_epoch, time.time() - start))
        # if test_miou_epoch>max_miou_pre:
        #     max_miou_pre = test_miou_epoch
        #     model.save_weights(config.path_savedmodel+'/unet_mobilenetv2/weights_epoch_%d'%(epoch+1))

        '''---visualize the results---'''
        if epoch%20 == 0:
            figure = plot_dset_one(model, test_dset.take(1), \
                        i_patch=np.random.randint(8), binary=False, \
                        multiscale=True, weight=False)


In [6]:
## training
train_loops(tra_dset_dist, test_dset_dist, epochs=200)


TypeError: Inputs to a layer should be tensors. Got: PerReplica:{
  0: tf.Tensor(
[[[[0.6200799  0.45047766 0.6109032  0.49702972]
   [0.63449526 0.35282668 0.6195055  0.355333  ]
   [0.4722984  0.4315957  0.59727067 0.30518308]
   ...
   [0.62156916 0.51233846 0.6774574  0.38955602]
   [0.61946267 0.3803712  0.6243561  0.48660418]
   [0.57406944 0.5501763  0.7096416  0.5246348 ]]

  [[0.5731365  0.49619794 0.7276622  0.5051314 ]
   [0.53605354 0.38739836 0.6072549  0.5007934 ]
   [0.4940034  0.20950058 0.6089326  0.33804658]
   ...
   [0.43885306 0.45036268 0.59669507 0.5086227 ]
   [0.67249584 0.43728137 0.67402637 0.46411133]
   [0.46359062 0.45490137 0.7083179  0.38257778]]

  [[0.64405715 0.47959828 0.6886976  0.5809803 ]
   [0.5436801  0.39817566 0.75190455 0.5559174 ]
   [0.54790485 0.41492885 0.74833465 0.51595914]
   ...
   [0.57650906 0.36684358 0.60683197 0.43213516]
   [0.6041667  0.5679051  0.66766006 0.5440774 ]
   [0.6530315  0.5631654  0.5873621  0.55319875]]

  ...

  

In [5]:
# %load_ext tensorboard
# !kill 6006
# %tensorboard --logdir logs/tensorb/
# http://localhost:16006


In [6]:
# model saving and loading
# path_weight = root_dir + '/models/temporal/UNet_gru_triple/weights_epoch_40'
# path_save_model = root_dir + '/models/pretrained/triple_backbone'
# model.save(path_save_model)
# model.save_weights(path_model)
# model = tf.keras.models.load_model(path_model)  ## load model
# model.load_weights(path_weight)

